# **Исследование надёжности заёмщиков**

**Описание проекта**


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.


Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных:
- **children** — количество детей в семье
- **days_employed** — общий трудовой стаж в днях
- **dob_years** — возраст клиента в годах
- **education** — уровень образования клиента
- **education_id** — идентификатор уровня образования
- **family_status** — семейное положение
- **family_status_id** — идентификатор семейного положения
- **gender** — пол клиента
- **income_type** — тип занятости
- **debt** — имел ли задолженность по возврату кредитов
- **total_income** — ежемесячный доход
- **purpose** — цель получения кредита

# Подключение библиотек

In [1]:
import numpy as np
import pandas as pd
from nltk.stem import SnowballStemmer as Snow_ball_Stemmer
from collections import Counter as Coll_counter

# Шаг 1. Общая информация и замечания по данным.

## 1.1. Изучение общей информации.

In [2]:
df = pd.read_csv("datasets/data.csv")

In [3]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Количество NA значений по столбцам.

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах **days_employed** и **total_income** одинаковое количество NA значений. Возможно в одних и тех же строках, надо будет проверить.

In [6]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


## 1.2. Ближе изучаем данные, находим замечания.

### 1.2.1.

In [7]:
df["children"].value_counts().sort_index()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: count, dtype: int64

В столбце "children" присутствуют странные значения -1 и 20. 20 детей очень большая редкость, 76 значений слишком много для реального случая. Это не единичные случаи, возможно это ошибка при занесении данных в таблицу или выгрузке. Исправим -1 на 0, а в числе 20 уберём лишний ноль. 

Наша гипотеза потвердилась, NA значения в этих двух столбцах в одних и тех же строках.

### 1.2.2.

In [8]:
df[(df["days_employed"].isna() == True) & (df["total_income"].isna() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


В столбце **days_employed** тип данных float64, стаж не может быть дробным числом, заменим на int64.

In [9]:
(df[df["days_employed"] < 0].shape[0]) / (len(df["days_employed"]))

0.7389547038327526

74% данных в столбце **days_employed** это отрицательные значения. Это не выбросы, это ошибка в данных, скорее всего при автоматической записи данных, отнимались отработанные дни из общего числа дней даты увольнения.

In [10]:
df["income_type"].value_counts()

income_type
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: count, dtype: int64

In [11]:
df[df["days_employed"] < 0]["income_type"].value_counts()

income_type
сотрудник          10014
компаньон           4577
госслужащий         1312
студент                1
предприниматель        1
в декрете              1
Name: count, dtype: int64

В эти отрицательные значения не входят пенсионеры и безработные. Можно взять значения по модулю.

In [12]:
df.groupby("income_type")["days_employed"].describe()

,count,mean,std,min,25%,50%,75%,max
income_type,,,,,,,,
безработный,2.0,366413.652744,40855.478519,337524.466835,351969.059790,366413.652744,380858.245699,395302.838654
в декрете,1.0,-3296.759962,NaN,-3296.759962,-3296.759962,-3296.759962,-3296.759962,-3296.759962
госслужащий,1312.0,-3399.896902,2788.371363,-15193.032201,-4759.399260,-2689.368353,-1257.171811,-39.954170
компаньон,4577.0,-2111.524398,2048.448594,-17615.563266,-2876.648520,-1547.382223,-685.687432,-30.195337
пенсионер,3443.0,365003.491245,21069.606065,328728.720605,346649.346146,365213.306266,383231.396871,401755.400475
предприниматель,1.0,-520.848083,NaN,-520.848083,-520.848083,-520.848083,-520.848083,-520.848083
сотрудник,10014.0,-2326.499216,2307.924129,-18388.949901,-3108.123025,-1574.202821,-746.027361,-24.141633
студент,1.0,-578.751554,NaN,-578.751554,-578.751554,-578.751554,-578.751554,-578.751554


У пенсионеров и безработных стаж работы варируется от 900 до 1100 лет, что нереалистично, но они в своих группах, и не будут воздействовать на других работников. Да и наше исследование не предполагает изучение влияния стажа.

Находим распределение NA значений по типу занятости.

In [13]:
df[df["days_employed"].isna() == True]["income_type"].value_counts()

income_type
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: count, dtype: int64

NA значения появились скорее всего из-за ошибки, а не потому-что данные отсутствуют, это кредитный отдел банка, он обязан собирать такую информацию. NA значения распределены по всем типам занятости, значит будем брать среднее для каждой группы чтобы заполнить NA.

### 1.2.3.

In [14]:
df["dob_years"].value_counts().sort_index()

dob_years
0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: count, dtype: int64

В столбце **dob_years** есть значения возраста 0, возможно данные просто были не внесены в базу, исключаем то что эти данные не собирались, это кредитный отдел банка, они обязанны собирать такую информацию.

In [15]:
df[df["dob_years"] == 0]["income_type"].value_counts()

income_type
сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: count, dtype: int64

Нулевые значения распределенны по категориям столбца **income_type**, значит и средние значения для замены нулей будем брать по этим категориям.

### 1.2.4.

In [16]:
df["education"].value_counts()

education
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: count, dtype: int64

В столбце **education** разный регистр, приведём всё к нижнему.

In [17]:
df["education_id"].value_counts().sort_index()

education_id
0     5260
1    15233
2      744
3      282
4        6
Name: count, dtype: int64

### 1.2.5.

In [18]:
df["family_status"].value_counts()

family_status
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: count, dtype: int64

В столбце **family_status** разный регистр, приведём всё к нижнему.

In [19]:
df["family_status_id"].value_counts().sort_index()

family_status_id
0    12380
1     4177
2      960
3     1195
4     2813
Name: count, dtype: int64

### 1.2.6.

In [20]:
df["gender"].value_counts()

gender
F      14236
M       7288
XNA        1
Name: count, dtype: int64

В столбце **gender** есть одно непонятное значение XNA, единичное неправильное значение не будет влияет на результат, оставим как есть.

### 1.2.7.

In [21]:
df["total_income"]

0        253875.639453
1        112080.014102
2        145885.952297
3        267628.550329
4        158616.077870
             ...      
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, Length: 21525, dtype: float64

Тип данных столбца **total_income** float64, текущая точность нам не нужна, заменим на int64.

In [22]:
df["total_income"].isna().sum()

2174

2174 значения NA в столбце **total_income**. Надо посмотреть как эти значения распределяются по категориям из столбца **income_type**.

In [23]:
df[df["total_income"].isna()]["income_type"].value_counts()

income_type
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: count, dtype: int64

Строки с NA значениями есть в разных категориях, поэтому брать среднее для замены пропусков будем по этим категориям.

### 1.2.8.

In [24]:
df["purpose"].value_counts()

purpose
свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образовани

В столбце **purpose** одинаковые цели кредита написанны разными словами. Надо провести лемматизацию по этому столбцу, чтобы выяснить уникальные цели кредита, после надо будет сделать категоризацию в отдельном столбце.

## Вывод (Что надо исправить):
| <span style="color:orange">1. Странные значения</span> | <span style="color:orange">2. Пропуски</span> | <span style="color:orange">3. Замена типа данных</span> | <span style="color:orange">4. Удаление дубликатов</span> | <span style="color:orange">5. Стемминг</span> | <span style="color:orange">6. Категоризация</span> |
| - | - | - | - | - | - |
| **children**: Поменять -1 на 0, а 20 на 2. | **days_employed**: вычислить среднее для каждой категории в столбце **income_type** и заполнить NA. | **days_employed**: поменять тип данных на int64. | После того как будет выполнена основная очистка и предобработка данных (пункты 1-3), разобраться с дубликатами в таблице. | В столбце **purpose** с помощью стемминга найти уникальные и самые популярные цели кредита. | С помощью лемм найденых в столбце **purpose** сделаем категоризацию целей кредита в новом столбце. |
| **days_employed**: поменять на значения по модулю. | **total_income**: вычислить среднее для каждой категории в столбце **income_type** и заполнить NA. | **total_income**: поменять тип данных на int64. |
| **dob_years** замена 0 значений средним, вычисленным исходя из категорий столбца **income_type**. |
| **education**: привести к нижнему регистру. |
| **family_status**: привести к нижнесму регистру. |

# Шаг 2. Очистка и предобработка данных.

## 2.1. Странные значения.

**children** меняем -1 на 0, а 20 на 2.

In [25]:
df["children"].replace(-1, 0, inplace=True)
df["children"].replace(20, 2, inplace=True)

In [26]:
df["children"].value_counts().sort_index()

children
0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: count, dtype: int64

Проверили, значения -1 и 20, исчезли.

**days_employed**: поменять на значения по модулю.

In [27]:
df["days_employed"] = df["days_employed"].abs()

In [28]:
len(df[df["days_employed"] < 0])

0

Отрицательных значений стажа больше нет.

**dob_years** замена 0 значений средним, вычисленным исходя из категорий столбца **income_type**.

In [29]:
df["dob_years"] = df.groupby("income_type")["dob_years"].transform(lambda x: x.replace(0, int(x.mean())))

In [30]:
df[df["dob_years"] == 0].shape[0]

0

Проверили, нулевых значений нет.

**education** и **family_status**: привести к нижнему регистру.

In [31]:
df["education"] = df["education"].str.lower()
df["family_status"] = df["family_status"].str.lower()

In [32]:
df["education"].value_counts()

education
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: count, dtype: int64

In [33]:
df["family_status"].value_counts()

family_status
женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: count, dtype: int64

Проверили, везде применился нижний регистр.

## 2.2. Пропуски.

**days_employed** и **total_income**: вычислить среднее для каждой категории в столбце **income_type** и заполнить NA.

In [34]:
df["days_employed"] = df.groupby("income_type")["days_employed"].transform(lambda x: x.fillna(x.mean()))
df["total_income"] = df.groupby("income_type")["total_income"].transform(lambda x: x.fillna(x.mean()))

In [35]:
df["days_employed"].isna().sum()

0

In [36]:
df["total_income"].isna().sum()

0

Все пропуски в столбцах заменены на среднее.

## 2.3. Замена типа данных.

**days_employed** и **total_income**: поменять тип данных на int64.

In [37]:
df["days_employed"] = df["days_employed"].astype(dtype="int64")
df["total_income"] = df["total_income"].astype(dtype="int64")

In [38]:
df["days_employed"].dtype

dtype('int64')

In [39]:
df["total_income"].dtype

dtype('int64')

Проверили, типы данных в этих столбцах поменялись на int64.

## 2.4. Удаление дубликатов.

Смотрим количество дубликатов в данных.

In [40]:
df.duplicated().sum()

71

In [41]:
df[df.duplicated()].sort_values(by=["total_income", "days_employed"])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3290,0,365003,58,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу
4851,0,365003,60,среднее,1,гражданский брак,1,F,пенсионер,0,137127,свадьба
5557,0,365003,58,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу
7808,0,365003,57,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
7921,0,365003,64,высшее,0,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
15991,0,2111,51,среднее,1,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
17379,0,2111,54,высшее,0,женат / замужем,0,M,компаньон,0,202417,операции с коммерческой недвижимостью
17774,1,2111,40,среднее,1,гражданский брак,1,F,компаньон,0,202417,строительство жилой недвижимости
19369,0,2111,45,среднее,1,гражданский брак,1,F,компаньон,0,202417,свадьба


71 дубликат. Скорее всего тут ошибочное повторное внесение данных. Избавимся от них, потому что дубликаты вызывают смещение наших финальных оценок. 

In [42]:
df.drop_duplicates(inplace=True)

In [43]:
df.duplicated().sum()

0

Избавились от дубликатов.

## 2.5. Лемматизация.

В столбце **purpose** с помощью стемминга ищем уникальные и самые популярные цели кредита, чтобы впоследствии сделать новый столбец с категориями целей кредита.

In [44]:
russian_stemmer = Snow_ball_Stemmer("russian")

In [45]:
stemm_list = []

for i in df["purpose"]:
    for j in i.split():
        stemm_list.append(russian_stemmer.stem(j))

popul_purp = Coll_counter(stemm_list)

Возмём топ-7 целей кредита.

In [46]:
sort_popul_purp = sorted(popul_purp.items(), key=lambda pair: pair[1], reverse=True)
most_pop_purp = [i for i, j in sort_popul_purp if len(i) > 1 and i != " " and i != "\n"]
most_pop_purp[0:7]

['недвижим', 'покупк', 'жил', 'образован', 'автомобил', 'операц', 'свадьб']

**недвижим** и **жил** это одна и та же цель. **покупк** и **операц** выкинем это не цели, а часть предложения. Создадим словарь, где ключами будут стеммы а значениями название категории.

In [47]:
purp_dict = {'недвижим': "недвижимость", 'жил': "недвижимость", 'образован': "образование", 'автомобил': "автомобиль",'свадьб': "свадьба"}
purp_dict

{'недвижим': 'недвижимость',
 'жил': 'недвижимость',
 'образован': 'образование',
 'автомобил': 'автомобиль',
 'свадьб': 'свадьба'}

## 2.6. Категоризация.

### 2.6.1 категоризация **purpose**.

Создадим в таблице столбец **purpose_cat**, на основе стемм которые получили ранее, в котором в каждой строке будет категория цели кредита.

In [48]:
def make_purp_cat(row):
    for k in purp_dict:
        if k in row["purpose"]:
            return purp_dict[k]
            break

In [49]:
df["purpose_cat"] = df.apply(make_purp_cat, axis=1)

Проверяем как прошла категоризация, есть ли ошибки в новом столбце.

In [50]:
df[["purpose", "purpose_cat"]]

,purpose,purpose_cat
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,покупка жилья,недвижимость
3,дополнительное образование,образование
4,сыграть свадьбу,свадьба
...,...,...
21520,операции с жильем,недвижимость
21521,сделка с автомобилем,автомобиль
21522,недвижимость,недвижимость
21523,на покупку своего автомобиля,автомобиль


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21454 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
 12  purpose_cat       21454 non-null  object
dtypes: int64(7), object(6)
memory usage: 2.3+ MB


In [52]:
df["purpose_cat"].value_counts()

purpose_cat
недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: count, dtype: int64

Категоризация прошла успешно.

### 2.6.2. Категоризация **children**.

Сделаем новый столбец **children_cat** с категориями по количеству детей. Разобьем следующим образом:
- 0 детей - нет детей.
- 1-2 ребёнка - 1-2 ребенка.
- больше 2 - многодетные.

In [53]:
def make_child_cat(col):
    if col == 0:
        return "нет детей"
    elif col <= 2:
        return "1-2 ребенка"
    else:
        return "многодетные"

In [54]:
df["children_cat"] = df["children"].apply(make_child_cat)

Проверяем.

In [55]:
df[["children", "children_cat"]]

,children,children_cat
0,1,1-2 ребенка
1,1,1-2 ребенка
2,0,нет детей
3,3,многодетные
4,0,нет детей
...,...,...
21520,1,1-2 ребенка
21521,0,нет детей
21522,1,1-2 ребенка
21523,3,многодетные


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21454 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
 12  purpose_cat       21454 non-null  object
 13  children_cat      21454 non-null  object
dtypes: int64(7), object(7)
memory usage: 2.5+ MB


In [57]:
df["children_cat"].value_counts()

children_cat
нет детей      14138
1-2 ребенка     6936
многодетные      380
Name: count, dtype: int64

Категоризация прошла успешно.

### 2.6.3. Категоризация total_income.

In [58]:
df["total_income"].describe()

count    2.145400e+04
mean     1.674316e+05
std      9.806060e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.518870e+05
75%      2.024170e+05
max      2.265604e+06
Name: total_income, dtype: float64

У нас ежемесячный доход от 20_000 до 2_265_604. Категоризуем так:
- меньше 100_000 - низкий доход.
- от 100_000 до 200_000 - средний доход.
- выше 200_000 - высокий доход.

In [59]:
def make_total_income_cat(col):
    if col < 100_000:
        return "низкий доход"
    elif 100_000 <= col <= 200_000:
        return "средний доход"
    else:
        return "высокий доход"

In [60]:
df["total_income_cat"] = df["total_income"].apply(make_total_income_cat)

Проверяем.

In [61]:
df[["total_income", "total_income_cat"]]

,total_income,total_income_cat
0,253875,высокий доход
1,112080,средний доход
2,145885,средний доход
3,267628,высокий доход
4,158616,средний доход
...,...,...
21520,224791,высокий доход
21521,155999,средний доход
21522,89672,низкий доход
21523,244093,высокий доход


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21454 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
 12  purpose_cat       21454 non-null  object
 13  children_cat      21454 non-null  object
 14  total_income_cat  21454 non-null  object
dtypes: int64(7), object(8)
memory usage: 2.6+ MB


In [63]:
df["total_income_cat"].value_counts()

total_income_cat
средний доход    11423
высокий доход     5568
низкий доход      4463
Name: count, dtype: int64

Категоризация прошла успешно.

# Шаг 3. Ответы на вопросы.

## 3.1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [64]:
depen_debt_child = df.pivot_table(index="children_cat", values="debt", aggfunc=["sum", "count"])
depen_debt_child.columns = ["num_of_debtors", "total_number_of_people"]
depen_debt_child["percent_of_debtors"] = ((depen_debt_child["num_of_debtors"] / depen_debt_child["total_number_of_people"]))
depen_debt_child["percent_of_debtors"] = depen_debt_child["percent_of_debtors"].apply(lambda x: f'''{(x * 100):.2f} %''')
depen_debt_child.sort_values(by="percent_of_debtors", ascending=False)

,num_of_debtors,total_number_of_people,percent_of_debtors
children_cat,,,
1-2 ребенка,646,6936,9.31 %
многодетные,31,380,8.16 %
нет детей,1064,14138,7.53 %


**Вывод**:  
- Самая большая задолженость по кредитам у людей с 1-2 детьми, у них же среднее число кредитов.
- Самая низкая задолженость у людей без детей, они же берут большее число кредитов.

## 3.2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [65]:
df["family_status"].value_counts()

family_status
женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: count, dtype: int64

In [66]:
depen_fam_status = df.pivot_table(index="family_status", values="debt", aggfunc=["sum", "count"])
depen_fam_status.columns = ["num_of_debtors", "total_number_of_people"]
depen_fam_status["percent_of_debtors"] = ((depen_fam_status["num_of_debtors"] / depen_fam_status["total_number_of_people"]))
depen_fam_status["percent_of_debtors"] = depen_fam_status["percent_of_debtors"].apply(lambda x: f'''{(x * 100):.2f} %''')
depen_fam_status.sort_values(by="percent_of_debtors", ascending=False)

,num_of_debtors,total_number_of_people,percent_of_debtors
family_status,,,
не женат / не замужем,274,2810,9.75 %
гражданский брак,388,4151,9.35 %
женат / замужем,931,12339,7.55 %
в разводе,85,1195,7.11 %
вдовец / вдова,63,959,6.57 %


**Вывод**:  
- Самый высокий процент должников среди людей которые не состоят в браке или состоят в гражданском браке (не официальном), у них примерно одинаковый процент задолжености. У них среднее количество кредитов.
- У женатых людей процент задолжености средний, но он не сильно выше тех кто не состоит в браке или овдовел. Зато у них самое большое количество кредитов.
- Самый низкий процент задолжености среди людей в разводе или овдовевших, но у них очень низкое количество взятых кредитов.

## 3.3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [68]:
depen_total_income = df.pivot_table(index="total_income_cat", values="debt", aggfunc=["sum", "count"])
depen_total_income.columns = ["num_of_debtors", "total_number_of_people"]
depen_total_income["percent_of_debtors"] = ((depen_total_income["num_of_debtors"] / depen_total_income["total_number_of_people"]))
depen_total_income["percent_of_debtors"] = depen_total_income["percent_of_debtors"].apply(lambda x: f'''{(x * 100):.2f} %''')
depen_total_income.sort_values(by="percent_of_debtors", ascending=False)

,num_of_debtors,total_number_of_people,percent_of_debtors
total_income_cat,,,
средний доход,999,11423,8.75 %
низкий доход,354,4463,7.93 %
высокий доход,388,5568,6.97 %


**Вывод**:  
- Самый больщой процент задолжености у людей со средним доходом, но у них самое большое количество кредитов.
- Самый низкий процент задолжености у людей с высоким доходом, и средним количеством кредитов.

## 3.4. Как разные цели кредита влияют на его возврат в срок?

In [70]:
df.sample()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat,children_cat,total_income_cat
20956,0,6169,35,среднее,1,женат / замужем,0,M,сотрудник,0,92957,дополнительное образование,образование,нет детей,низкий доход


In [77]:
depen_purpose_cat = df.pivot_table(index="purpose_cat", values="debt", aggfunc=["sum", "count"])
depen_purpose_cat.columns = ["num_of_debtors", "total_number_of_people"]
depen_purpose_cat["percent_of_debtors"] = ((depen_purpose_cat["num_of_debtors"] / depen_purpose_cat["total_number_of_people"]))
depen_purpose_cat["percent_of_debtors"] = depen_purpose_cat["percent_of_debtors"].apply(lambda x: f'''{(x * 100):.2f} %''')
depen_purpose_cat.sort_values(by="percent_of_debtors", ascending=False)

,num_of_debtors,total_number_of_people,percent_of_debtors
purpose_cat,,,
автомобиль,403,4306,9.36 %
образование,370,4013,9.22 %
свадьба,186,2324,8.00 %
недвижимость,782,10811,7.23 %


**Вывод**:
- Самый большой процент задолжености у тех кто берёт кредит на машину, примерно такой же процент у тех кто берёт кредит на образование. Так же у них примерно равное количество кредитов, это средний показатель среди всех тех кто брал кредиты.
- Самый низкий процент задолжености у тех кто берёт кредит на недвижимость, у них же самое большое количество кредитов. 

# **Общий вывод**:

Отвечая на поставленный вопрос - влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок? Выяснилось что влияет:
- Заемщики без детей, состоящие в официальном браке, или когда то бывшие в официальном браке - в целом более <span style="color:green">надёжные, ответственные заёмщики</span>
- Заемщики с 1-2 детьми, не состоящие в официальном браке или состоящие в гражданском браке - в целом менее <span style="color:red">надёжные, ответственные заёмщики</span>

Вот общий вывод учитывая выводы по всем вопросам:
| <span style="color:green">надёжные, ответственные заёмщики</span> | <span style="color:red">менее надёжные и ответственные заёмщики</span> |
|-|-|
| Нет детей, в браке или бывшие в браке, с высоким доходом, берущие кредит на недвижимость или свадьбу. | 1-2 ребёнка, не в браке или в гражданском браке, со средним доходом, берущие кредит на автомобиль или образование. |